In [1]:
%install_ext https://github.com/mkrphys/ipython-tikzmagic/blob/master/tikzmagic.py

/home/pi/Documents/Github/venv/local/lib/python2.7/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed tikzmagic.py. To use it, type:
  %load_ext tikzmagic


In [2]:
import notebook.nbextensions
notebook.nbextensions.install_nbextension('https://raw.githubusercontent.com/ipython-contrib/IPython-notebook-extensions/master/nbextensions/config/main.js', user=True)

u'/home/pi/.local/share/jupyter/nbextensions/main.js'

In [3]:
%%javascript
Jupyter.utils.load_extensions('main')

<IPython.core.display.Javascript object>

In [4]:
from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('notebook', {"load_extensions": {"config": True}})

{'load_extensions': {'config': True}}

In [5]:
%tikz

ERROR: Line magic function `%tikz` not found.


In [6]:
import os
 
mypath = !ipython locate
print os.path.dirname(mypath[0]) # output >> 'C:\Users\annahambi'

/home/pi


In [7]:
%%file C:/Users/PAULINKENBRANDT/.ipython/nbextensions/toc.css
 
/*extracted from https://gist.github.com/magican/5574556*/
#toc {
  overflow-y: scroll;
  max-height: 300px;
 
  ol.nested {
    counter-reset: item;
    list-style: none;
  }
  li.nested {
    display: block;
  }
  li.nested:before {
    counter-increment: item;
    content: counters(item, ".")" ";
  }
}
 
#toc-wrapper {
  position: fixed;
  top: 120px;
  max-width:430px;
  right: 20px;
  border: thin solid rgba(0, 0, 0, 0.38);
  border-radius: 5px;
  padding:10px;
  background-color: #fff;
  opacity: .8;
  z-index: 100;
}
 
#toc-wrapper.closed {
  min-width: 100px;
  width: auto;
  transition: width;
}
#toc-wrapper:hover{
  opacity: 1;
}
#toc-wrapper .header {
  font-size: 18px;
  font-weight: bold;
}
#toc-wrapper .hide-btn {
  font-size: 14px;
  font-family: monospace;
}

Writing C:/Users/PAULINKENBRANDT/.ipython/nbextensions/toc.css


IOError: [Errno 2] No such file or directory: 'C:/Users/PAULINKENBRANDT/.ipython/nbextensions/toc.css'

In [6]:
%%file C:/Users/PAULINKENBRANDT/.ipython/nbextensions/toc.js
define(["require"], function (require) {
  "use strict";
 
  var clone_anchor = function (element) {
    // clone link
    var h = element.find("div.text_cell_render").find(':header').first();
    var a = h.find('a').clone();
    var new_a = $("<a>");
    new_a.attr("href", a.attr("href"));
    // get the text *excluding* the link text, whatever it may be
    var hclone = h.clone();
    hclone.children().remove();
    new_a.text(hclone.text());
    return new_a;
  };
 
  var ol_depth = function (element) {
    // get depth of nested ol
    var d = 0;
    while (element.prop("tagName").toLowerCase() == 'ol') {
      d += 1;
      element = element.parent();
    }
    return d;
  };
  
  var create_toc_div = function () {
    var toc_wrapper = $('<div id="toc-wrapper"/>')
    .append(
      $("<div/>")
      .addClass("header")
      .text("Contents ")
      .click( function(){
        console.log(this);
        $('#toc').slideToggle();
        $('#toc-wrapper').toggleClass('closed');
        if ($('#toc-wrapper').hasClass('closed')){
          $('#toc-wrapper .hide-btn').text('[+]');
        } else {
          $('#toc-wrapper .hide-btn').text('[-]');
        }
        return false;
      }).append(
        $("<a/>")
        .attr("href", "#")
        .addClass("hide-btn")
        .text("[-]")
      )
    ).append(
        $("<div/>").attr("id", "toc")
    );
    $("body").append(toc_wrapper);
  };
 
  var table_of_contents = function (threshold) {
    if (threshold === undefined) {
      threshold = 4;
    }
    var cells = IPython.notebook.get_cells();
      
    var toc_wrapper = $("#toc-wrapper");
    if (toc_wrapper.length === 0) {
      create_toc_div();
    }
  
    var ol = $("<ol/>");
    $("#toc").empty().append(ol);
  
    for (var i=0; i < cells.length; i++) {
      var cell = cells[i];
    
      if (cell.cell_type !== 'heading') continue;
    
      var level = cell.level;
      if (level > threshold) continue;
    
      var depth = ol_depth(ol);
 
      // walk down levels
      for (; depth < level; depth++) {
        var new_ol = $("<ol/>");
        ol.append(new_ol);
        ol = new_ol;
      }
      // walk up levels
      for (; depth > level; depth--) {
        ol = ol.parent();
      }
      //
      ol.append(
        $("<li/>").append(clone_anchor(cell.element))
      );
    }
 
    $(window).resize(function(){
      $('#toc').css({maxHeight: $(window).height() - 200});
    });
 
    $(window).trigger('resize');
  };
    
  var toggle_toc = function () {
    // toggle draw (first because of first-click behavior)
    $("#toc-wrapper").toggle();
    // recompute:
    table_of_contents();
  };
  
  var toc_button = function () {
    if (!IPython.toolbar) {
      $([IPython.events]).on("app_initialized.NotebookApp", toc_button);
      return;
    }
    if ($("#toc_button").length === 0) {
      IPython.toolbar.add_buttons_group([
        {
          'label'   : 'Table of Contents',
          'icon'    : 'icon-list',
          'callback': toggle_toc,
          'id'      : 'toc_button'
        },
      ]);
    }
  };
  
  var load_css = function () {
    var link = document.createElement("link");
    link.type = "text/css";
    link.rel = "stylesheet";
    link.href = require.toUrl("./toc.css");
    console.log(link);
    document.getElementsByTagName("head")[0].appendChild(link);
  };
  
  var load_ipython_extension = function () {
    load_css();
    toc_button();
    $([IPython.events]).on("notebook_loaded.Notebook", table_of_contents);
    console.log('Table of Contents extension loaded');
  };
 
  return {
    load_ipython_extension : load_ipython_extension,
    toggle_toc : toggle_toc,
    table_of_contents : table_of_contents,
    
  };
 
});


Writing C:/Users/PAULINKENBRANDT/.ipython/nbextensions/toc.js


In [5]:
%%file C:/Users/PAULINKENBRANDT/.ipython/nbextensions/custom.js   
 
require(["/nbextensions/toc.js"], function (toc) {
console.log('Table of Contents extension loaded');
toc.load_ipython_extension();
$([IPython.events]).on("notebook_loaded.Notebook", toc.table_of_contents);
});
 
IPython.load_extensions('toc');

Overwriting C:/Users/PAULINKENBRANDT/.ipython/nbextensions/custom.js
